## Transferlearning

In [ ]:
import torch
from torch.nn import functional as F
import torchvision.models as models
from torchvision.models import ResNet18_Weights
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch import optim, nn

from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

%load_ext autoreload
%autoreload 2
import utils

# GPUの設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model

In [ ]:
num_classes = 10
model.fc = nn.Linear(512, num_classes)

# パラメータの重みを凍結
for name, param in model.named_parameters():
    if not name.startswith('fc'):
        param.requires_grad = False

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CIFAR10(root='./cifar10_data', train=True, download=True, transform=transform)
val_dataset = CIFAR10(root='./cifar10_data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False, num_workers=4)

# モデルをGPUに移動
model = model.to(device)

opt = optim.Adam(model.parameters(), lr=0.03)

# resnetは大きいのでCUDAが必要
train_losses, val_losses, val_accuracies, losses_in_epoch = utils.learn(model, train_loader, val_loader, opt, F.cross_entropy, 3)